In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd())) 
import logging 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.features.load_games_from_db import load_games_from_db


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
path = '../data/processed_game_odds.csv'
df_odds = pd.read_csv(path)


In [4]:
df_odds.shape

(13691, 8)

In [5]:
df_games = load_games_from_db()

2025-07-21 21:57:04,253 [INFO] Successully loaded 10983 games from database


There are 10983 games for  modeling...


In [12]:
scored_data_path = '../src/scoring/output.csv'
df_scored = pd.read_csv(scored_data_path)

In [21]:
def keep_preferred_bookmaker(group, preferred_bookmakers):

    # Filter to same_time and preferred bookmaker
    same_time_bookmaker_preferred = group[(group['time_match']==1) & (group['bookmaker'].isin(preferred_bookmakers))]

    # Filter to preferred bookmaker
    bookmaker_preferred = group[group['bookmaker'].isin(preferred_bookmakers)]

    if len(same_time_bookmaker_preferred) > 0:
        # Keep first preferred bookmaker where time matches
        return same_time_bookmaker_preferred.iloc[[0]]
    
    elif len(same_time_bookmaker_preferred) == 0 and len(bookmaker_preferred) > 0: 
        # Keep first preferred bookmaker
        return bookmaker_preferred.iloc[[0]]
    
    else:
        # If no preferred bookmaker, keep all rows for this game_id
        return group


def join_odds_to_scored_data(df_scored: pd.DataFrame, df_odds: pd.DataFrame) -> pd.DataFrame:

    logging.info(f"Earliest Date in Scored Data: {df_scored['game_date'].min()}, Latest Date in Scored Data: {df_scored['game_date'].max()}")
    logging.info(f"Earliest Date in Odds Data: {df_odds['game_date'].min()}, Latest Date in Odds Data: {df_odds['game_date'].max()}")

    # Convert to datetime
    df_scored['game_date_time'] = pd.to_datetime(df_scored['game_date_time'], utc=True)
    df_scored['game_date'] = pd.to_datetime(df_scored['game_date'], utc=True)
    df_odds['game_date'] = pd.to_datetime(df_odds['game_date'], utc=True)
    df_odds['commence_time'] = pd.to_datetime(df_odds['commence_time'], utc=True)
    df_odds['timestamp'] = pd.to_datetime(df_odds['timestamp'], utc=True)

    # Join odds data to scored data
    odds_cols = ['timestamp','game_date','commence_time','home_team','away_team','bookmaker','home_odds','away_odds']
    logging.info(f"df_score Shape BEFORE Join: {df_scored.shape}")
    logging.info(f"df_scored # Unique Games BEFORE Join: {df_scored['game_id'].nunique()}")
    df_scored = df_scored.merge(
        df_odds[odds_cols],
        how='left',
        on=['game_date','home_team','away_team'], # will produce duplicate rows for double-headers (ok for now...)
        suffixes=('','_odds'),
        indicator=True
        )
    logging.info(f"df_score Shape AFTER Join: {df_scored.shape}")
    logging.info(f"df_scored # Unique Games AFTER Join: {df_scored['game_id'].nunique()}")

    logging.info(f"Join Match Rate: {df_scored['_merge'].value_counts(dropna=False, normalize=True).round(3)}")
    df_scored.drop('_merge', axis=1, inplace=True)

    # Dedup based on approximate times and selecting preferred bookmaker
    print(f"game_date_time: {df_scored['game_date_time'].iloc[0]}, dtype: {df_scored['game_date_time'].dtype}")
    print(f"commence_time: {df_scored['commence_time'].iloc[0]}, dtype: {df_scored['commence_time'].dtype}")
        
    df_scored['time_diff'] = (df_scored['game_date_time']-df_scored['commence_time']).dt.total_seconds()/60 # difference in minutes
    df_scored['time_match'] = (df_scored['time_diff'].abs() < 10).astype(int) # times within 10 minutes of each other are the same
    
    PREFERRED_BOOKMAKERS = ["betmgm", "draftkings", "fanduel", "williamhill_us", "espnbet"]
    df_scored['preferred_bookmaker'] = df_scored['bookmaker'].isin(PREFERRED_BOOKMAKERS).astype(int)

    df_scored = (
        df_scored
        .groupby('game_id')
        .apply(keep_preferred_bookmaker, PREFERRED_BOOKMAKERS, include_groups=False)
        .reset_index()
        .drop('level_1', axis=1)
    )

    logging.info(f"df_scored Shape AFTER Deduplication: {df_scored.shape}")
    logging.info(f"df_scored # Unique Games AFTER Join: {df_scored['game_id'].nunique()}")

    return df_scored

In [14]:
df_scored[['game_id','game_date','game_date_time']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10983 entries, 0 to 10982
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   game_id         10983 non-null  int64         
 1   game_date       10983 non-null  datetime64[ns]
 2   game_date_time  10983 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 257.5+ KB


In [22]:
df_scored = join_odds_to_scored_data(df_scored=df_scored, df_odds=df_odds)

2025-07-21 22:07:34,286 [INFO] Earliest Date in Scored Data: 2021-04-01 00:00:00+00:00, Latest Date in Scored Data: 2025-07-21 00:00:00+00:00
2025-07-21 22:07:34,289 [INFO] Earliest Date in Odds Data: 2021-04-01 00:00:00+00:00, Latest Date in Odds Data: 2025-07-20 00:00:00+00:00
2025-07-21 22:07:34,396 [INFO] df_score Shape BEFORE Join: (10983, 195)
2025-07-21 22:07:34,397 [INFO] df_scored # Unique Games BEFORE Join: 10983
2025-07-21 22:07:34,488 [INFO] df_score Shape AFTER Join: (14005, 201)
2025-07-21 22:07:34,489 [INFO] df_scored # Unique Games AFTER Join: 10983
2025-07-21 22:07:34,490 [INFO] Join Match Rate: _merge
both          0.855
left_only     0.145
right_only    0.000
Name: proportion, dtype: float64


game_date_time: 2021-04-01 17:05:00+00:00, dtype: datetime64[ns, UTC]
commence_time: 2021-04-01 17:05:00+00:00, dtype: datetime64[ns, UTC]


2025-07-21 22:07:44,313 [INFO] df_scored Shape AFTER Deduplication: (10983, 203)
2025-07-21 22:07:44,313 [INFO] df_scored # Unique Games AFTER Join: 10983


In [26]:
df_games['game_date'].min(), df_games['game_date'].max() 

(Timestamp('2021-04-01 00:00:00'), Timestamp('2025-07-21 00:00:00'))

In [27]:
df_odds['game_date'].min(), df_odds['game_date'].max()

(Timestamp('2021-04-01 00:00:00'), Timestamp('2025-07-20 00:00:00'))

In [8]:
df_odds.sort_values('commence_time', inplace=True)

In [9]:
df_odds[df_odds['game_date']=='2025-06-20']

,timestamp,game_date,home_team,away_team,commence_time,bookmaker,home_odds,away_odds
8084,2025-06-19T12:55:37Z,2025-06-20,Oakland Athletics,Houston Astros,2025-06-20T02:06:00Z,fanduel,110,-130
8085,2025-06-19T12:55:37Z,2025-06-20,Los Angeles Dodgers,San Diego Padres,2025-06-20T02:11:00Z,fanduel,-250,205
5829,2025-06-20T12:55:37Z,2025-06-20,Chicago Cubs,Seattle Mariners,2025-06-20T18:21:00Z,fanduel,-134,116
5830,2025-06-20T12:55:37Z,2025-06-20,Pittsburgh Pirates,Texas Rangers,2025-06-20T22:41:00Z,fanduel,154,-184
5831,2025-06-20T12:55:37Z,2025-06-20,New York Yankees,Baltimore Orioles,2025-06-20T23:06:00Z,fanduel,-225,188
5832,2025-06-20T12:55:37Z,2025-06-20,Tampa Bay Rays,Detroit Tigers,2025-06-20T23:06:00Z,fanduel,-110,-106
5833,2025-06-20T12:55:37Z,2025-06-20,Toronto Blue Jays,Chicago White Sox,2025-06-20T23:07:00Z,fanduel,-205,172
5834,2025-06-20T12:55:37Z,2025-06-20,Miami Marlins,Atlanta Braves,2025-06-20T23:11:00Z,fanduel,122,-144
5835,2025-06-20T12:55:37Z,2025-06-20,Philadelphia Phillies,New York Mets,2025-06-20T23:15:00Z,williamhill_us,-190,158
5836,2025-06-20T12:55:37Z,2025-06-20,St. Louis Cardinals,Cincinnati Reds,2025-06-20T23:16:00Z,fanduel,-142,120


In [10]:
df_games['game_date'] = pd.to_datetime(df_games['game_date'])
df_games[df_games['game_date'] ==pd.Timestamp('2025-06-20')]

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type
10626,777436,2025-06-20,2025-06-20 18:20:00+00:00,112,136,Chicago Cubs,Seattle Mariners,4.0,9.0,Final,Wrigley Field,R
10627,777432,2025-06-20,2025-06-20 22:40:00+00:00,134,140,Pittsburgh Pirates,Texas Rangers,2.0,6.0,Final,PNC Park,R
10628,777431,2025-06-20,2025-06-20 23:05:00+00:00,139,116,Tampa Bay Rays,Detroit Tigers,14.0,8.0,Final,George M. Steinbrenner Field,R
10629,777438,2025-06-20,2025-06-20 23:05:00+00:00,147,110,New York Yankees,Baltimore Orioles,3.0,5.0,Final,Yankee Stadium,R
10630,777435,2025-06-20,2025-06-20 23:07:00+00:00,141,145,Toronto Blue Jays,Chicago White Sox,1.0,7.0,Final,Rogers Centre,R
10631,777439,2025-06-20,2025-06-20 23:10:00+00:00,146,144,Miami Marlins,Atlanta Braves,6.0,2.0,Final,loanDepot park,R
10632,777433,2025-06-20,2025-06-20 23:15:00+00:00,138,113,St. Louis Cardinals,Cincinnati Reds,6.0,1.0,Final,Busch Stadium,R
10633,777437,2025-06-20,2025-06-20 23:15:00+00:00,143,121,Philadelphia Phillies,New York Mets,10.0,2.0,Final,Citizens Bank Park,R
10634,777434,2025-06-20,2025-06-21 00:10:00+00:00,142,158,Minnesota Twins,Milwaukee Brewers,6.0,17.0,Final,Target Field,R
10635,777430,2025-06-20,2025-06-21 00:40:00+00:00,115,109,Colorado Rockies,Arizona Diamondbacks,8.0,14.0,Final,Coors Field,R


In [11]:
df_games['game_date'] = pd.to_datetime(df_games['game_date'])
df_odds['game_date'] = pd.to_datetime(df_odds['game_date'])
df_odds['commence_time'] = pd.to_datetime(df_odds['commence_time'], utc=True)
df_odds['timestamp'] = pd.to_datetime(df_odds['timestamp'], utc=True)

In [28]:
def keep_preferred_bookmaker(group, preferred_bookmakers):
    # Filter to same_time == 1 and preferred bookmakers
    same_time_preferred = group[(group['time_match'] == 1) & 
                               (group['bookmaker'].isin(preferred_bookmakers))]

    in_preferred_bookmaker = group[group['bookmaker'].isin(preferred_bookmakers)]
    
    if len(same_time_preferred) > 0:
        # Keep the first preferred bookmaker where time matches
        return same_time_preferred.iloc[[0]]
    elif len(same_time_preferred) == 0 and len(in_preferred_bookmaker) > 0:
        # Keep the first preferred bookmaker
        return in_preferred_bookmaker.iloc[[0]]
    else:
        # If no preferred bookmaker with same_time==1, keep all rows for this game_id
        return group



In [29]:
df_games_m = df_games.merge(df_odds[['timestamp','game_date','commence_time','home_team','away_team','bookmaker', 'home_odds','away_odds']],
                          how='left',
                          left_on=['game_date', 'home_team', 'away_team'],
                          right_on=['game_date','home_team','away_team'],
                          suffixes=('','_odds'), 
                            indicator=True
                         )

In [14]:
df_games.shape, df_games_m.shape

((10983, 12), (13987, 18))

In [30]:
df_games_m['time_diff'] = (df_games_m['game_date_time'] - df_games_m['commence_time']).dt.total_seconds()/60
df_games_m['time_match'] = (df_games_m['time_diff'].abs() < 10).astype(int)
PREFERRED_BOOKMAKERS = ["betmgm", "draftkings", "fanduel", "williamhill_us", "espnbet"]
df_games_m['preferred_bookmaker'] = df_games_m['bookmaker'].isin(PREFERRED_BOOKMAKERS).astype(int)

In [31]:
df_dedup = df_games_m.groupby('game_id').apply(keep_preferred_bookmaker, PREFERRED_BOOKMAKERS, include_groups=False).reset_index().drop('level_1', axis=1)

In [32]:
df_dedup.shape

(10983, 21)

In [33]:
df_dedup['game_id'].nunique()

10983

In [34]:
df_dedup.head(2)

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type,timestamp,commence_time,bookmaker,home_odds,away_odds,_merge,time_diff,time_match,preferred_bookmaker
0,632169,2021-04-10,2021-04-10 20:05:00+00:00,137,115,San Francisco Giants,Colorado Rockies,4.0,3.0,Final,Oracle Park,R,2021-04-10 12:55:00+00:00,2021-04-10 20:05:00+00:00,williamhill_us,-179.0,162.0,both,0.0,1,1
1,632170,2021-04-11,2021-04-11 18:10:00+00:00,145,118,Chicago White Sox,Kansas City Royals,3.0,4.0,Final,Guaranteed Rate Field,R,2021-04-11 12:55:00+00:00,2021-04-11 18:10:00+00:00,williamhill_us,-130.0,120.0,both,0.0,1,1


In [35]:
df_dedup['_merge'].value_counts(dropna=False), df_dedup['_merge'].value_counts(dropna=False, normalize=True)

(_merge
 both          9003
 left_only     1980
 right_only       0
 Name: count, dtype: int64,
 _merge
 both          0.819721
 left_only     0.180279
 right_only    0.000000
 Name: proportion, dtype: float64)

In [13]:
df_games_m[['game_id','game_date','game_date_time','commence_time','home_team','away_team','home_odds','away_odds','bookmaker', '_merge']].head()

,game_id,game_date,game_date_time,commence_time,home_team,away_team,home_odds,away_odds,bookmaker,_merge
0,634642,2021-04-01,2021-04-01 17:05:00+00:00,2021-04-01 17:05:00+00:00,New York Yankees,Toronto Blue Jays,-185.0,165.0,betmgm,both
1,634645,2021-04-01,2021-04-01 17:10:00+00:00,2021-04-01 17:10:00+00:00,Detroit Tigers,Cleveland Indians,165.0,-189.0,betmgm,both
2,634638,2021-04-01,2021-04-01 18:10:00+00:00,2021-04-01 18:10:00+00:00,Milwaukee Brewers,Minnesota Twins,-110.0,100.0,betmgm,both
3,634634,2021-04-01,2021-04-01 18:20:00+00:00,2021-04-01 18:20:00+00:00,Chicago Cubs,Pittsburgh Pirates,-208.0,190.0,williamhill_us,both
4,634622,2021-04-01,2021-04-01 19:05:00+00:00,2021-04-01 19:05:00+00:00,Philadelphia Phillies,Atlanta Braves,-115.0,105.0,betmgm,both


In [14]:
df_games_m['_merge'].value_counts(dropna=False)

_merge
both          12007
left_only      1965
right_only        0
Name: count, dtype: int64

In [38]:
df_games_m['_merge'].value_counts(dropna=False, normalize=True)[:-1].round(3)

_merge
both         0.858
left_only    0.142
Name: proportion, dtype: float64

In [60]:
df_games_m['season'] = df_games_m['game_date'].dt.year
df_games_m.groupby('season')['_merge'].value_counts(dropna=False)

season  _merge    
2021    both          2104
        left_only      532
        right_only       0
2022    both          2309
        left_only      425
        right_only       0
2023    both          3638
        left_only      413
        right_only       0
2024    both          2471
        left_only      335
        right_only       0
2025    both          1485
        left_only      260
        right_only       0
Name: count, dtype: int64

In [61]:
df_games_m.groupby('season')['_merge'].value_counts(dropna=False, normalize=True)

season  _merge    
2021    both          0.798179
        left_only     0.201821
        right_only    0.000000
2022    both          0.844550
        left_only     0.155450
        right_only    0.000000
2023    both          0.898050
        left_only     0.101950
        right_only    0.000000
2024    both          0.880613
        left_only     0.119387
        right_only    0.000000
2025    both          0.851003
        left_only     0.148997
        right_only    0.000000
Name: proportion, dtype: float64

In [17]:
df_games_m[(df_games_m['game_date_time'] != df_games_m['commence_time']) & (df_games_m['_merge']=='both')].shape

(6824, 18)

In [16]:
df_games_m[(df_games_m['game_date_time'] != df_games_m['commence_time']) & (df_games_m['_merge']=='both')].head(10)

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type,timestamp,commence_time,bookmaker,home_odds,away_odds,_merge
15,634631,2021-04-02,2021-04-03 01:38:00+00:00,108,145,Los Angeles Angels,Chicago White Sox,8.0,12.0,Final,Angel Stadium,R,2021-04-01 12:55:00+00:00,2021-04-02 02:05:00+00:00,betmgm,105.0,-115.0,both
16,634605,2021-04-02,2021-04-03 01:40:00+00:00,133,117,Oakland Athletics,Houston Astros,5.0,9.0,Final,Oakland Coliseum,R,2021-04-01 12:55:00+00:00,2021-04-02 02:07:00+00:00,betmgm,-105.0,-105.0,both
18,634578,2021-04-02,2021-04-03 02:10:00+00:00,136,137,Seattle Mariners,San Francisco Giants,3.0,6.0,Final,T-Mobile Park,R,2021-04-01 12:55:00+00:00,2021-04-02 02:10:00+00:00,betmgm,-105.0,-105.0,both
19,634644,2021-04-03,2021-04-03 17:05:00+00:00,147,141,New York Yankees,Toronto Blue Jays,5.0,3.0,Final,Yankee Stadium,R,2021-04-03 12:55:00+00:00,2021-04-03 17:06:00+00:00,betmgm,-175.0,145.0,both
20,634577,2021-04-03,2021-04-03 17:10:00+00:00,111,110,Boston Red Sox,Baltimore Orioles,2.0,4.0,Final,Fenway Park,R,2021-04-03 12:55:00+00:00,2021-04-03 17:11:00+00:00,williamhill_us,-172.0,157.0,both
21,634573,2021-04-03,2021-04-03 17:10:00+00:00,116,114,Detroit Tigers,Cleveland Indians,5.0,2.0,Final,Comerica Park,R,2021-04-03 12:55:00+00:00,2021-04-03 17:11:00+00:00,betmgm,145.0,-175.0,both
23,634643,2021-04-03,2021-04-03 18:20:00+00:00,112,134,Chicago Cubs,Pittsburgh Pirates,5.0,1.0,Final,Wrigley Field,R,2021-04-03 12:55:00+00:00,2021-04-03 18:21:00+00:00,williamhill_us,-161.0,150.0,both
24,634580,2021-04-03,2021-04-03 20:05:00+00:00,143,144,Philadelphia Phillies,Atlanta Braves,4.0,0.0,Final,Citizens Bank Park,R,2021-04-03 12:55:00+00:00,2021-04-03 20:06:00+00:00,williamhill_us,101.0,-111.0,both
25,634629,2021-04-03,2021-04-03 20:07:00+00:00,133,117,Oakland Athletics,Houston Astros,1.0,9.0,Final,Oakland Coliseum,R,2021-04-02 12:55:00+00:00,2021-04-03 01:41:00+00:00,williamhill_us,-141.0,130.0,both
27,634654,2021-04-03,2021-04-03 20:10:00+00:00,113,138,Cincinnati Reds,St. Louis Cardinals,9.0,6.0,Final,Great American Ball Park,R,2021-04-03 12:55:00+00:00,2021-04-03 20:11:00+00:00,betmgm,-115.0,-105.0,both


In [ ]:
df_games_m['time_diff'] = (df_games_m['game_date_time'] - df_games_m['commence_time']).dt.total_seconds()/60
df_games_m['time_match'] = (df_games_m['time_diff'].abs() < 10).astype(int)
PREFERRED_BOOKMAKERS = ["betmgm", "draftkings", "fanduel", "williamhill_us", "espnbet"]
df_games_m['preferred_bookmaker'] = np.where(df_games_m['bookmaker'].isin(PREFERRED_BOOKMAKERS), 1, 0)

In [48]:
cols = ['game_id','game_date','game_date_time','commence_time','time_diff','time_match','home_team','away_team','bookmaker','preferred_bookmaker','home_odds','away_odds','_merge']
df_games_m[df_games_m['game_id'].duplicated(keep=False)][cols].head(12)

,game_id,game_date,game_date_time,commence_time,time_diff,time_match,home_team,away_team,bookmaker,preferred_bookmaker,home_odds,away_odds,_merge
25,634629,2021-04-03,2021-04-03 20:07:00+00:00,2021-04-03 01:41:00+00:00,1106.0,0,Oakland Athletics,Houston Astros,williamhill_us,1,-141.0,130.0,both
26,634629,2021-04-03,2021-04-03 20:07:00+00:00,2021-04-03 20:07:00+00:00,0.0,1,Oakland Athletics,Houston Astros,williamhill_us,1,110.0,-120.0,both
42,634557,2021-04-04,2021-04-04 19:10:00+00:00,2021-04-04 00:11:00+00:00,1139.0,0,Colorado Rockies,Los Angeles Dodgers,betmgm,1,190.0,-250.0,both
43,634557,2021-04-04,2021-04-04 19:10:00+00:00,2021-04-04 19:11:00+00:00,-1.0,1,Colorado Rockies,Los Angeles Dodgers,betmgm,1,190.0,-250.0,both
45,634572,2021-04-04,2021-04-04 20:10:00+00:00,2021-04-04 00:41:00+00:00,1169.0,0,San Diego Padres,Arizona Diamondbacks,fanduel,1,-213.0,180.0,both
46,634572,2021-04-04,2021-04-04 20:10:00+00:00,2021-04-04 20:11:00+00:00,-1.0,1,San Diego Padres,Arizona Diamondbacks,williamhill_us,1,-213.0,193.0,both
63,634586,2021-04-06,2021-04-06 20:07:00+00:00,2021-04-06 01:38:00+00:00,1109.0,0,Los Angeles Angels,Houston Astros,williamhill_us,1,-115.0,105.0,both
64,634586,2021-04-06,2021-04-06 20:07:00+00:00,2021-04-06 20:07:00+00:00,0.0,1,Los Angeles Angels,Houston Astros,williamhill_us,1,105.0,-115.0,both
81,634583,2021-04-07,2021-04-07 18:05:00+00:00,2021-04-07 00:05:00+00:00,1080.0,0,Texas Rangers,Toronto Blue Jays,williamhill_us,1,120.0,-130.0,both
82,634583,2021-04-07,2021-04-07 18:05:00+00:00,2021-04-07 18:06:00+00:00,-1.0,1,Texas Rangers,Toronto Blue Jays,williamhill_us,1,170.0,-189.0,both


In [62]:
dups = df_games_m[df_games_m.duplicated(['game_id','preferred_bookmaker'],keep=False)][cols]

In [63]:
len(dups)

5332

In [64]:
dups['game_id'].nunique()

2359

In [65]:
len(dups[dups['time_match']==1]), dups[dups['time_match']==1]['game_id'].nunique()

(3533, 2228)

In [66]:
df_match = dups[dups['time_match']==1]

In [68]:
len(df_match), df_match['game_id'].nunique()

(3533, 2228)

In [67]:
df_match.groupby(['game_id','game_date_time'])['preferred_bookmaker'].agg('max').reset_index()['preferred_bookmaker'].value_counts(dropna=False)

preferred_bookmaker
1    2228
Name: count, dtype: int64

In [54]:
df_match[df_match.duplicated('game_id',keep=False)].head(10)

,game_id,game_date,game_date_time,commence_time,time_diff,time_match,home_team,away_team,bookmaker,preferred_bookmaker,home_odds,away_odds,_merge
144,632207,2021-04-11,2021-04-11 23:08:00+00:00,2021-04-11 23:08:00+00:00,0.0,1,Atlanta Braves,Philadelphia Phillies,mybookieag,0,-192.0,125.0,both
145,632207,2021-04-11,2021-04-11 23:08:00+00:00,2021-04-11 23:09:00+00:00,-1.0,1,Atlanta Braves,Philadelphia Phillies,williamhill_us,1,-156.0,147.0,both
297,634389,2021-04-23,2021-04-23 23:05:00+00:00,2021-04-23 23:05:00+00:00,0.0,1,Baltimore Orioles,Oakland Athletics,mybookieag,0,260.0,-370.0,both
298,634389,2021-04-23,2021-04-23 23:05:00+00:00,2021-04-23 23:06:00+00:00,-1.0,1,Baltimore Orioles,Oakland Athletics,williamhill_us,1,120.0,-130.0,both
299,634412,2021-04-23,2021-04-23 23:10:00+00:00,2021-04-23 23:10:00+00:00,0.0,1,Detroit Tigers,Kansas City Royals,mybookieag,0,145.0,-192.0,both
300,634412,2021-04-23,2021-04-23 23:10:00+00:00,2021-04-23 23:11:00+00:00,-1.0,1,Detroit Tigers,Kansas City Royals,williamhill_us,1,105.0,-115.0,both
301,634429,2021-04-23,2021-04-23 23:10:00+00:00,2021-04-23 23:10:00+00:00,0.0,1,New York Mets,Washington Nationals,mybookieag,0,-286.0,204.0,both
302,634429,2021-04-23,2021-04-23 23:10:00+00:00,2021-04-23 23:11:00+00:00,-1.0,1,New York Mets,Washington Nationals,williamhill_us,1,-303.0,260.0,both
304,634413,2021-04-23,2021-04-23 23:10:00+00:00,2021-04-23 23:10:00+00:00,0.0,1,Tampa Bay Rays,Toronto Blue Jays,mybookieag,0,440.0,-833.0,both
305,634413,2021-04-23,2021-04-23 23:10:00+00:00,2021-04-23 23:11:00+00:00,-1.0,1,Tampa Bay Rays,Toronto Blue Jays,williamhill_us,1,-154.0,143.0,both


In [102]:
df_deduplicated = df_games_m.groupby('game_id').apply(keep_preferred_bookmaker,include_groups=False).reset_index()

In [103]:
df_deduplicated.shape

(10968, 23)

In [97]:
df_deduplicated['game_id'].nunique()

10968

In [105]:
df_deduplicated[df_deduplicated.duplicated('game_id',keep=False)][cols].describe()

,game_id,game_date,time_diff,time_match,preferred_bookmaker,home_odds,away_odds
count,0.0,0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaT,NaN,NaN,NaN,NaN,NaN
min,NaN,NaT,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaT,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaT,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaT,NaN,NaN,NaN,NaN,NaN
max,NaN,NaT,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df_games_m[(df_games_m['game_id']==634029)][cols]

,game_id,game_date,game_date_time,commence_time,time_diff,time_match,home_team,away_team,bookmaker,preferred_bookmaker,home_odds,away_odds,_merge
695,634029,2021-05-19,2021-05-20 00:07:00+00:00,2021-05-19 01:38:00+00:00,1349.0,0,Los Angeles Angels,Cleveland Indians,mybookieag,0,-141.0,120.0,both
696,634029,2021-05-19,2021-05-20 00:07:00+00:00,2021-05-19 01:39:00+00:00,1348.0,0,Los Angeles Angels,Cleveland Indians,betmgm,1,-145.0,120.0,both


In [39]:
def convert_odds_to_probability(odds: int) -> float:
    """Convert moneyline odds to implied probability"""
    if odds < 0:
        probability = abs(odds) / (100 + abs(odds))
    else:
        probability = 100 / (100 + odds)
    
    return probability

In [41]:
df_dedup.head(2)

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type,timestamp,commence_time,bookmaker,home_odds,away_odds,_merge,time_diff,time_match,preferred_bookmaker
0,632169,2021-04-10,2021-04-10 20:05:00+00:00,137,115,San Francisco Giants,Colorado Rockies,4.0,3.0,Final,Oracle Park,R,2021-04-10 12:55:00+00:00,2021-04-10 20:05:00+00:00,williamhill_us,-179.0,162.0,both,0.0,1,1
1,632170,2021-04-11,2021-04-11 18:10:00+00:00,145,118,Chicago White Sox,Kansas City Royals,3.0,4.0,Final,Guaranteed Rate Field,R,2021-04-11 12:55:00+00:00,2021-04-11 18:10:00+00:00,williamhill_us,-130.0,120.0,both,0.0,1,1


In [51]:
df_dedup['home_implied_prob'] = df_dedup['home_odds'].apply(convert_odds_to_probability)

In [52]:
df_dedup.head(2)

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type,timestamp,commence_time,bookmaker,home_odds,away_odds,_merge,time_diff,time_match,preferred_bookmaker,home_implied_prob
0,632169,2021-04-10,2021-04-10 20:05:00+00:00,137,115,San Francisco Giants,Colorado Rockies,4.0,3.0,Final,Oracle Park,R,2021-04-10 12:55:00+00:00,2021-04-10 20:05:00+00:00,williamhill_us,-179.0,162.0,both,0.0,1,1,0.641577
1,632170,2021-04-11,2021-04-11 18:10:00+00:00,145,118,Chicago White Sox,Kansas City Royals,3.0,4.0,Final,Guaranteed Rate Field,R,2021-04-11 12:55:00+00:00,2021-04-11 18:10:00+00:00,williamhill_us,-130.0,120.0,both,0.0,1,1,0.565217


In [50]:
float(convert_odds_to_probability(df_dedup['home_odds'].iloc[:3]))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().